In [1]:
import pandas as pd

In [2]:
xls = pd.ExcelFile('data.xls')
df1 = pd.read_excel(xls, 'Export')
df2 = pd.read_excel(xls, 'Sheet 2')
data = pd.concat([df1, df2])
data = data.reset_index(drop=True)
data.loc[1]['comment']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


'اصلا خوب نبود قطعاتش داخله سوراخایی که بود جا نمیشد سوراخا خیلی کوچیک بودن و میتونم بگم که با زور جا انداختمش اصلا خوب نبود،،بنظرم ریلی بود کشوش خیلی بهتر بود چون الان کجو کلست،،حالا در کل منکه گرفتم،اما پیشنهادش نمیکنم.'

In [3]:
from hazm import *
normalizer = Normalizer()
# Select Verified Comments
data = data[data['verification_status'] == 'verified']

# Combine Features ('title', 'comment', 'advantages', 'disadvantages')
data['combined'] = data['title']+'. '+data['comment']+'. '+data['advantages']+'. '+data['disadvantages']

# Remove other columns except this ones
data = data[['combined', 'label', 'title', 'comment', 'advantages', 'disadvantages']]

# Normalize each column
data['comment'] = data['comment'].apply(normalizer.normalize)
data['title'] = data['title'].apply(normalizer.normalize)
data['advantages'] = data['advantages'].apply(normalizer.normalize)
data['disadvantages'] = data['disadvantages'].apply(normalizer.normalize)
data['combined'] = data['combined'].apply(normalizer.normalize)

data.loc[1]['comment']

'اصلا خوب نبود قطعاتش داخله سوراخایی که بود جا نمیشد سوراخا خیلی کوچیک بودن و میتونم بگم که با زور جا انداختمش اصلا خوب نبود،، بنظرم ریلی بود کشوش خیلی بهتر بود چون الان کجو کلست،، حالا در کل منکه گرفتم، اما پیشنهادش نمیکنم.'

In [4]:
import string
import re
import hazm as hz

def remove_mentions(input_text):
    return re.sub(r'@\w+', '', input_text)
def remove_urls(input_text):
    return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
def emoji_oneword(input_text):
    return input_text.replace('_','')
def remove_punctuation(input_text):
    punct = string.punctuation + '«»'
    trantab = str.maketrans(punct, len(punct)*' ')
    return input_text.translate(trantab)
def remove_digits(input_text):
    return re.sub('\d+', '', input_text)
def remove_english(input_text):
#     return input_text
    return re.sub('[a-zA-Z0-9?><;,{}[\]\-_+=!@#$%\^&*|\']*' ,'', input_text)
def remove_stopwords(input_text):
    return input_text
#     stopwords_list = hz.stopwords_list()
#     words = input_text.split() 
#     clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 1] 
#     return " ".join(clean_words) 
def stemming(input_text):
    stemmer = Stemmer()
    words = input_text.split() 
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)
def transform(data):
    clean_data = data.apply(remove_mentions).apply(remove_urls).apply(emoji_oneword).apply(remove_punctuation).apply(remove_digits).apply(remove_english).apply(remove_stopwords).apply(stemming)
    return clean_data

In [5]:
data['combined'] =  transform(data.combined)
data = data.dropna()
data.loc[0]

combined         پس میده متاسفانه من چون همیشه مولفیکس استفاده ...
label                                                            2
title                                             پس میده متاسفانه
comment          من چون همیشه مولفیکس استفاده کردم و هیچوقت مشک...
advantages                                                    None
disadvantages                                                 None
Name: 0, dtype: object

In [6]:
data.loc[1]

combined         اصلا خوب نبود اصلا خوب نبود قطعات داخله سوراخا...
label                                                            2
title                                               اصلا خوب نبود.
comment          اصلا خوب نبود قطعاتش داخله سوراخایی که بود جا ...
advantages                                                    None
disadvantages                                                 None
Name: 1, dtype: object

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)


In [8]:
from sklearn.model_selection import train_test_split
data = data[['combined', 'label']]
train, test = train_test_split(data, test_size=0.4)

In [9]:
train_vectors = vectorizer.fit_transform(train['combined'])
test_vectors = vectorizer.transform(test['combined'])

In [10]:
len(vectorizer.get_feature_names())

8057

In [12]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(probability=True, kernel='linear', gamma='auto')
t0 = time.time()
classifier_linear.fit(train_vectors, train['label'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1


In [13]:
# results

report = classification_report(test['label'], prediction_linear, output_dict=True)
print('positive: ', report['2.0'])
print('negative: ', report['0.0'])
print('neutral: ', report['1.0'])
report

positive:  {'precision': 0.8198903313492993, 'recall': 0.9283591594141372, 'f1-score': 0.8707598118513725, 'support': 18844}
negative:  {'precision': 0.7362494073020389, 'recall': 0.7704999379729562, 'f1-score': 0.7529853912832636, 'support': 8061}
negative:  {'precision': 0.39568345323741005, 'recall': 0.018290655138011307, 'f1-score': 0.03496503496503497, 'support': 3007}


{'0.0': {'precision': 0.7362494073020389,
  'recall': 0.7704999379729562,
  'f1-score': 0.7529853912832636,
  'support': 8061},
 '1.0': {'precision': 0.39568345323741005,
  'recall': 0.018290655138011307,
  'f1-score': 0.03496503496503497,
  'support': 3007},
 '2.0': {'precision': 0.8198903313492993,
  'recall': 0.9283591594141372,
  'f1-score': 0.8707598118513725,
  'support': 18844},
 'accuracy': 0.7943300347686547,
 'macro avg': {'precision': 0.6506077306295828,
  'recall': 0.5723832508417016,
  'f1-score': 0.5529034126998904,
  'support': 29912},
 'weighted avg': {'precision': 0.7547051357345822,
  'recall': 0.7943300347686547,
  'f1-score': 0.7549997657729846,
  'support': 29912}}

In [14]:
review = """
اصلا خوب نبود بنظرم میتونست خیلی بهتر باشه
"""
import pandas as pd
review = pd.Series([review])
review = transform(review)
print(review.loc[0], "\n\n")
review_vector = vectorizer.transform([review.loc[0]]) # vectorizing
print(classifier_linear.predict(review_vector))

اصلا خوب نبود بنظر میتونس خیل به باشه 


[0.]


In [15]:
review = """
کاملا راضیم
"""
import pandas as pd
review = pd.Series([review])
review = transform(review)
print(review.loc[0], "\n\n")
review_vector = vectorizer.transform([review.loc[0]]) # vectorizing
print(classifier_linear.predict(review_vector))

کاملا راض 


[2.]


In [17]:
review = """
این همون iPhone 7 یا حتی میشه گفت iPhone 6s هست که ارتقا دادنش واقعا چی داره که مردم برن آیفون های قبلی شون رو بدن اینو بگیرن با قیمت نجومی اونم تو سال 2017 که بیشتر برند ها بی حاشیه زدن حالا درسته که iPhone x بی حاشیه هست ولی اونم اشکالات خودش رو داره مثل بیرون زدگی دوربین با وجود ضخامت بیشتر نسبت به قبل و حذف سنسور اثر انگشت حتی سامسونگ هم با اون همه سنسور مختلف باز دوربین نزده بیرون یا به این اندازه ضخامت نداره
"""
import pandas as pd
review = pd.Series([review])
review = transform(review)
print(review.loc[0], "\n\n")
review_vector = vectorizer.transform([review.loc[0]]) # vectorizing
print(classifier_linear.predict(review_vector))

این همون یا حت میشه گف هس که ارتقا دادن واقعا چ داره که مرد برن آیفون  قبل شون رو بدن اینو بگیرن با قیم نجوم اون تو سال که ب برند  ب حاشیه زدن حالا درسته که ب حاشیه هس ول اون اشکال خود رو داره مثل بیرون زدگ دوربین با وجود ضخام ب نسب به قبل و حذف سنسور اثر انگ حت سامسونگ ه با اون همه سنسور مختلف باز دوربین نزده بیرون یا به این اندازه ضخام نداره 


[1.]


In [18]:
import joblib as jl

filename = 'classifier_linear.sav'
jl.dump(classifier_linear, filename)

filename = 'prediction_linear.sav'
jl.dump(prediction_linear, filename)

filename = 'test.sav'
jl.dump(test, filename)

filename = 'test_label.sav'
jl.dump(test['label'], filename)

filename = 'train.sav'
jl.dump(train, filename)


['classifier_linear.sav']